In [102]:
import pandas as pd

In [103]:
xls = pd.ExcelFile('../data.xlsx')

In [104]:
sheets = ['1', '10','16', '18', '21','25','39','41','49']

In [105]:
df = {}
for num in sheets:
    df[num] = pd.read_excel(xls, num) 

In [106]:
for key in df:
    df[key]['SyncDate'] = pd.to_datetime(df[key]['SyncDate'])
    df[key].sort_values(by='SyncDate', ascending=True, inplace=True)
    df[key]['SyncDate'] = df[key]['SyncDate'].dt.floor('H')
    df[key] = df[key].resample('H', on = 'SyncDate').mean()
    df[key] = df[key].interpolate(method='linear', limit_direction='both')
    df[key].drop_duplicates(inplace=True)

In [107]:
df_radon = pd.DataFrame(index=df['49'].index)

In [108]:
for key in df:
   print(key)
   df_radon[f'Radon_{key}'] = df[key]['Radon']

1
10
16
18
21
25
39
41
49


In [109]:
correlations_unwindowed_hourly = df_radon.corr()

In [110]:
correlations_unwindowed_hourly['Radon_16'].sort_values(ascending=False)

Radon_16    1.000000
Radon_49    0.626968
Radon_39    0.559502
Radon_10    0.218340
Radon_18    0.177222
Radon_21    0.071272
Radon_25   -0.112564
Radon_1    -0.130809
Radon_41   -0.158191
Name: Radon_16, dtype: float64

In [111]:
correlations_unwindowed_hourly['Radon_39'].sort_values(ascending=False)

Radon_39    1.000000
Radon_49    0.660898
Radon_16    0.559502
Radon_10    0.487265
Radon_18    0.251460
Radon_25    0.034619
Radon_41   -0.053990
Radon_1    -0.103221
Radon_21   -0.127083
Name: Radon_39, dtype: float64

In [112]:
df_radon = df_radon.resample('D').mean()

In [113]:
correlations_unwindowed_daily = df_radon.corr()

In [114]:
correlations_unwindowed_daily['Radon_16'].sort_values(ascending=False)

Radon_16    1.000000
Radon_49    0.651968
Radon_39    0.576449
Radon_10    0.228352
Radon_18    0.173371
Radon_21    0.065540
Radon_25   -0.114386
Radon_1    -0.152402
Radon_41   -0.164408
Name: Radon_16, dtype: float64

In [115]:
correlations_unwindowed_daily['Radon_39'].sort_values(ascending=False)

Radon_39    1.000000
Radon_49    0.676563
Radon_16    0.576449
Radon_10    0.523355
Radon_18    0.262037
Radon_25    0.049917
Radon_41   -0.049676
Radon_1    -0.123749
Radon_21   -0.137821
Name: Radon_39, dtype: float64

In [116]:
df_radon

,Radon_1,Radon_10,Radon_16,Radon_18,Radon_21,Radon_25,Radon_39,Radon_41,Radon_49
SyncDate,,,,,,,,,
2022-05-08,0.504167,0.727083,0.422917,0.235417,0.225000,0.560417,0.308333,0.470833,0.185417
2022-05-09,0.662500,0.625000,0.279167,0.295833,0.454167,0.287500,0.475000,0.333333,0.316667
2022-05-10,1.200000,0.725000,0.541667,0.787500,0.491667,0.445833,0.729167,0.497917,0.950000
2022-05-11,2.141667,1.191667,0.500000,1.300000,0.987500,0.950000,0.558333,1.022917,1.058333
2022-05-12,4.752083,1.195833,0.595833,0.804167,1.129167,0.762500,0.745833,0.912500,0.641667
...,...,...,...,...,...,...,...,...,...
2022-08-03,10.125000,1.062500,1.012500,0.854167,1.354167,7.047917,1.466667,23.170833,1.120833
2022-08-04,7.495833,1.208333,1.520833,1.916667,1.350000,10.400000,2.350000,17.962500,1.779167
2022-08-05,7.275000,1.500000,1.245833,2.668750,1.750000,9.670833,2.329167,16.854167,2.243750


In [121]:
from termcolor import colored
radon_16_counters = {'Radon_1':0, 'Radon_10':0, 'Radon_18':0, 'Radon_21':0, 'Radon_25':0, 'Radon_39':0, 'Radon_41':0, 'Radon_49':0}
radon_39_counters = {'Radon_1':0, 'Radon_10':0, 'Radon_18':0, 'Radon_21':0, 'Radon_25':0, 'Radon_16':0, 'Radon_41':0, 'Radon_49':0}
for i in range(0, df_radon.shape[0], 8):
    window_number = i // 8
    df_win = df_radon.iloc[i:i+8]
    print(window_number, f'{df_win.index[0]}-{df_win.index[-1]}')
    correlations_windowed_daily = df_win.corr()
    correlations_windowed_daily.fillna(0, inplace=True)
    print(correlations_windowed_daily['Radon_16'].sort_values(ascending=False))
    print()
    print('------------------------------------------------------------------------------------------------------------------')
    print()
    print(correlations_windowed_daily['Radon_39'].sort_values(ascending=False))
    print()
    print('------------------------------------------------------------------------------------------------------------------')
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    if correlations_windowed_daily['Radon_16']['Radon_1'] > 0.5:
        radon_16_counters['Radon_1'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_10'] > 0.5:
        radon_16_counters['Radon_10'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_18'] > 0.5:
        radon_16_counters['Radon_18'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_21'] > 0.5:
        radon_16_counters['Radon_21'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_25'] > 0.5:
        radon_16_counters['Radon_25'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_39'] > 0.5:
        radon_16_counters['Radon_39'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_41'] > 0.5:
        radon_16_counters['Radon_41'] += 1
    if correlations_windowed_daily['Radon_16']['Radon_49'] > 0.5:
        radon_16_counters['Radon_49'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_1'] > 0.5:
        radon_39_counters['Radon_1'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_10'] > 0.5:
        radon_39_counters['Radon_10'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_18'] > 0.5:
        radon_39_counters['Radon_18'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_21'] > 0.5:
        radon_39_counters['Radon_21'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_25'] > 0.5:
        radon_39_counters['Radon_25'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_16'] > 0.5:
        radon_39_counters['Radon_16'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_41'] > 0.5:
        radon_39_counters['Radon_41'] += 1
    if correlations_windowed_daily['Radon_39']['Radon_49'] > 0.5:
        radon_39_counters['Radon_49'] += 1

0 2022-05-08 00:00:00-2022-05-15 00:00:00
Radon_16    1.000000
Radon_49    0.971315
Radon_39    0.904119
Radon_41    0.584164
Radon_21    0.570203
Radon_10    0.496820
Radon_25    0.405742
Radon_18    0.277773
Radon_1     0.153338
Name: Radon_16, dtype: float64

------------------------------------------------------------------------------------------------------------------

Radon_39    1.000000
Radon_49    0.924150
Radon_16    0.904119
Radon_21    0.647184
Radon_41    0.600158
Radon_10    0.518292
Radon_25    0.285441
Radon_18    0.223479
Radon_1     0.108383
Name: Radon_39, dtype: float64

------------------------------------------------------------------------------------------------------------------
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
1 2022-05-16 00:00:00-2022-05-23 00:00:00
Radon_16    1.000000
Radon_25    0.215816
Radon_10    0.060382
Radon_18    0.035369
Radon_1    -0.037874
Radon_49   -0.068572


In [122]:
radon_16_counters

{'Radon_1': 0,
 'Radon_10': 2,
 'Radon_18': 5,
 'Radon_21': 4,
 'Radon_25': 0,
 'Radon_39': 7,
 'Radon_41': 4,
 'Radon_49': 6}

In [123]:
radon_39_counters

{'Radon_1': 2,
 'Radon_10': 7,
 'Radon_18': 2,
 'Radon_21': 3,
 'Radon_25': 2,
 'Radon_16': 7,
 'Radon_41': 2,
 'Radon_49': 10}